In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

driver_path = 'C:/Users/Landon Hawley/Downloads/chromedriver-win64/chromedriver.exe'
service = Service(driver_path)

driver = webdriver.Chrome(service=service)

# Open the URL
url = "https://www.espn.com/nfl/stats/player/_/season/2021/seasontype/2"
driver.get(url)

time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

# Get the page source after JavaScript has loaded
html_content = driver.page_source

driver.quit()

In [2]:
soup = BeautifulSoup(html_content, "lxml")

namerank_columns = ['Rank', 'Name/Team']
namerank_df = pd.DataFrame(columns = namerank_columns)

namerank_div = soup.find(name = 'div', attrs = {'class' : 'ResponsiveTable ResponsiveTable--fixed-left mt4 Table2__title--remove-capitalization'})
table = namerank_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    namerank_series = pd.Series(data, index = namerank_columns)
    namerank_df = pd.concat([namerank_df, namerank_series.to_frame().T], ignore_index=True)

In [3]:
import re


team_abbrs = [
    "ARI", "ATL", "BAL", "BUF", "CAR", "CHI", "CIN", "CLE", "DAL", "DEN",
    "DET", "GB", "HOU", "IND", "JAX", "KC", "LAC", "LAR", "LV", "MIA",
    "MIN", "NE", "NO", "NYG", "NYJ", "PHI", "PIT", "SEA", "SF", "TB",
    "TEN", "WSH", "OAK"
]


team_pattern = r'(' + '|'.join(team_abbrs) + r')$'


def split_string_first(s):
    match = re.search(team_pattern, s)
    if match:
        return s[:match.start()]
    return s


def split_string_second(s):
    match = re.search(team_pattern, s)
    if match:
        return match.group()
    return ''


namerank_df['Name'] = namerank_df['Name/Team'].apply(split_string_first)
namerank_df['Team'] = namerank_df['Name/Team'].apply(split_string_second)
namerank_df.drop('Name/Team', inplace=True, axis=1)

In [4]:
stats_columns = ['Position', 'Games Played', 'Completions', 'Passing Attempts', 'Completion Percentage', 'Passing Yards',
                 'Avg Yards per Pass Attempt', 'Passing Yards per Game', 'Longest Pass', 'Passing Touchdowns', 'Interceptions',
                 'Total Sacks', 'Sack Yards Lost', 'Adjusted QBR', 'Passer Rating']
stats_df = pd.DataFrame(columns = stats_columns)

stats_div = soup.find(name = 'div', attrs = {'class' : 'Table__Scroller'})
table = stats_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    stats_series = pd.Series(data, index = stats_columns)
    stats_df = pd.concat([stats_df, stats_series.to_frame().T], ignore_index=True)

stats_df

,Position,Games Played,Completions,Passing Attempts,Completion Percentage,Passing Yards,Avg Yards per Pass Attempt,Passing Yards per Game,Longest Pass,Passing Touchdowns,Interceptions,Total Sacks,Sack Yards Lost,Adjusted QBR,Passer Rating
0,QB,17,485,719,67.5,"5,316",7.4,312.7,62,43,12,22,144,73.1,102.1
1,QB,17,443,672,65.9,"5,014",7.5,294.9,72,38,15,31,214,70.9,97.7
2,QB,17,404,601,67.2,"4,886",8.1,287.4,79,41,17,30,243,69.2,102.9
3,QB,17,436,658,66.3,"4,839",7.4,284.6,75,37,13,28,146,67.7,98.5
4,QB,17,428,626,68.4,"4,804",7.7,282.6,61,23,14,40,241,58.4,94.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,RB,13,0,1,0.0,0,0.0,0.0,0,0,1,0,0,25.6,0.0
105,P,17,0,1,0.0,0,0.0,0.0,0,0,0,0,0,,39.6
106,WR,13,0,2,0.0,0,0.0,0.0,0,0,0,0,0,3.0,39.6
107,RB,13,0,1,0.0,0,0.0,0.0,0,0,0,0,0,0.6,39.6


In [5]:
total_stats = pd.concat([namerank_df, stats_df], axis=1)
total_stats

,Rank,Name,Team,Position,Games Played,Completions,Passing Attempts,Completion Percentage,Passing Yards,Avg Yards per Pass Attempt,Passing Yards per Game,Longest Pass,Passing Touchdowns,Interceptions,Total Sacks,Sack Yards Lost,Adjusted QBR,Passer Rating
0,1,Tom Brady,TB,QB,17,485,719,67.5,"5,316",7.4,312.7,62,43,12,22,144,73.1,102.1
1,2,Justin Herbert,LAC,QB,17,443,672,65.9,"5,014",7.5,294.9,72,38,15,31,214,70.9,97.7
2,3,Matthew Stafford,LAR,QB,17,404,601,67.2,"4,886",8.1,287.4,79,41,17,30,243,69.2,102.9
3,4,Patrick Mahomes,KC,QB,17,436,658,66.3,"4,839",7.4,284.6,75,37,13,28,146,67.7,98.5
4,5,Derek Carr,LV,QB,17,428,626,68.4,"4,804",7.7,282.6,61,23,14,40,241,58.4,94.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,86,David Montgomery,CHI,RB,13,0,1,0.0,0,0.0,0.0,0,0,1,0,0,25.6,0.0
105,86,Blake Gillikin,NO,P,17,0,1,0.0,0,0.0,0.0,0,0,0,0,0,,39.6
106,86,A.J. Brown,TEN,WR,13,0,2,0.0,0,0.0,0.0,0,0,0,0,0,3.0,39.6
107,86,D'Andre Swift,DET,RB,13,0,1,0.0,0,0.0,0.0,0,0,0,0,0,0.6,39.6


In [7]:
total_stats.drop(total_stats[total_stats['Name'].str.contains('/')].index, inplace = True)
total_stats

,Rank,Name,Team,Position,Games Played,Completions,Passing Attempts,Completion Percentage,Passing Yards,Avg Yards per Pass Attempt,Passing Yards per Game,Longest Pass,Passing Touchdowns,Interceptions,Total Sacks,Sack Yards Lost,Adjusted QBR,Passer Rating
0,1,Tom Brady,TB,QB,17,485,719,67.5,"5,316",7.4,312.7,62,43,12,22,144,73.1,102.1
1,2,Justin Herbert,LAC,QB,17,443,672,65.9,"5,014",7.5,294.9,72,38,15,31,214,70.9,97.7
2,3,Matthew Stafford,LAR,QB,17,404,601,67.2,"4,886",8.1,287.4,79,41,17,30,243,69.2,102.9
3,4,Patrick Mahomes,KC,QB,17,436,658,66.3,"4,839",7.4,284.6,75,37,13,28,146,67.7,98.5
4,5,Derek Carr,LV,QB,17,428,626,68.4,"4,804",7.7,282.6,61,23,14,40,241,58.4,94.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,86,David Montgomery,CHI,RB,13,0,1,0.0,0,0.0,0.0,0,0,1,0,0,25.6,0.0
105,86,Blake Gillikin,NO,P,17,0,1,0.0,0,0.0,0.0,0,0,0,0,0,,39.6
106,86,A.J. Brown,TEN,WR,13,0,2,0.0,0,0.0,0.0,0,0,0,0,0,3.0,39.6
107,86,D'Andre Swift,DET,RB,13,0,1,0.0,0,0.0,0.0,0,0,0,0,0,0.6,39.6


In [9]:
total_stats.to_csv("Passing_Stats_2021.csv", index = False)
print("Data saved to 2021_Passing_Stats.csv")

Data saved to 2021_Passing_Stats.csv
